https://www.manythings.org/anki/rus-eng.zip

In [29]:
%matplotlib inline

In [30]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import requests
from zipfile import ZipFile

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
with ZipFile('rus-eng.zip') as zip_file:
    zip_file.extractall()

In [32]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [33]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-Я.!?]+", r" ", s)
    return s

In [34]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    # Read the file and split into lines
    file_path = 'rus.txt'
    lines = open(file_path, encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[0:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [35]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [36]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 467119 sentence pairs
Trimmed to 27228 sentence pairs
Counting words...
Counted words:
rus 9939
eng 4229
['я не замужем .', 'i m not married .']


The Encoder
-----------





In [38]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------




In [39]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [40]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [41]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [42]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [43]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [44]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [45]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [46]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [47]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 58s (- 13m 37s) (5000 6%) 3.1117
1m 46s (- 11m 35s) (10000 13%) 2.6531
2m 35s (- 10m 21s) (15000 20%) 2.3476
3m 24s (- 9m 21s) (20000 26%) 2.1715
4m 12s (- 8m 25s) (25000 33%) 2.0113
5m 1s (- 7m 32s) (30000 40%) 1.8486
5m 50s (- 6m 40s) (35000 46%) 1.7492
6m 39s (- 5m 49s) (40000 53%) 1.6227
7m 28s (- 4m 58s) (45000 60%) 1.5603
8m 17s (- 4m 8s) (50000 66%) 1.5052
9m 6s (- 3m 18s) (55000 73%) 1.3950
9m 55s (- 2m 28s) (60000 80%) 1.3757
10m 44s (- 1m 39s) (65000 86%) 1.3036
11m 33s (- 0m 49s) (70000 93%) 1.1916
12m 22s (- 0m 0s) (75000 100%) 1.2107


In [48]:
evaluateRandomly(encoder1, decoder1)

> я очень рад тебя видеть .
= i m very glad to see you .
< i m very glad to see you . <EOS>

> мы в состоянии воины с япониеи .
= we re at war with japan .
< we re on with the war . <EOS>

> это я тебя спас .
= i m the one who saved you .
< i m the one who you you . <EOS>

> ты неотразима .
= you are irresistible .
< you re a . <EOS>

> я безработныи .
= i m unemployed .
< i m a . . <EOS>

> они все мертвы .
= they are all dead .
< they re all dead . <EOS>

> опять начинаешь старую песню .
= you re giving me the same old line .
< you re the smartest in the . . . <EOS>

> я им понравлюсь .
= they re going to like me .
< they re going . . <EOS>

> не уверен что понимаю .
= i m not sure i understand .
< i m not sure what i agree . <EOS>

> ты держишь его кверх ногами .
= you re holding it upside down .
< you re holding it upside down . <EOS>



## GRU - 2 рекуррентных слоя

In [58]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=num_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1*self.num_layers, 1, self.hidden_size, device=device)

In [59]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=num_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1*self.num_layers, 1, self.hidden_size, device=device)

In [60]:
hidden_size = 256
encoder2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder2 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder2, decoder2, 75000, print_every=5000)

1m 6s (- 15m 26s) (5000 6%) 3.1166
2m 1s (- 13m 11s) (10000 13%) 2.6594
3m 1s (- 12m 5s) (15000 20%) 2.3940
3m 57s (- 10m 53s) (20000 26%) 2.2153
4m 53s (- 9m 47s) (25000 33%) 2.0613
5m 49s (- 8m 44s) (30000 40%) 1.9465
6m 46s (- 7m 44s) (35000 46%) 1.7903
7m 52s (- 6m 53s) (40000 53%) 1.6704
9m 7s (- 6m 4s) (45000 60%) 1.6096
10m 11s (- 5m 5s) (50000 66%) 1.4906
11m 10s (- 4m 3s) (55000 73%) 1.4179
12m 9s (- 3m 2s) (60000 80%) 1.3614
13m 6s (- 2m 1s) (65000 86%) 1.3026
14m 5s (- 1m 0s) (70000 93%) 1.2265
15m 3s (- 0m 0s) (75000 100%) 1.1916


In [62]:
evaluateRandomly(encoder2, decoder2)

> боюсь вы напрасно теряете время .
= i m afraid you re wasting your time .
< i m afraid you re wasting your time . <EOS>

> я уверена что это может подождать .
= i m sure this can wait .
< i m sure that s wrong . <EOS>

> ты красивыи мужчина .
= you re a handsome man .
< you are a handsome man . <EOS>

> мы благодарны вам за помощь .
= we re grateful for your assistance .
< we re grateful for your help . <EOS>

> прошу прощения что побеспокоил .
= i m sorry to have bothered you .
< i m sorry to have bothered to . <EOS>

> ты обеспокоен .
= you re worried .
< you re worried . <EOS>

> он доволен своеи судьбои .
= he is contented with his lot .
< he is pleased with his his . <EOS>

> у меня проблема с концентрациеи .
= i m having trouble concentrating .
< i m having a a . <EOS>

> я не пущу вас за руль .
= i m not going to let you drive .
< i m not going to let you . . <EOS>

> мне очень жаль .
= i m really sorry .
< i m really sorry . <EOS>



## LSTM - 2 рекуррентных слоя

In [90]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(self.num_layers, 1, self.hidden_size, device=device),
                torch.zeros(self.num_layers, 1, self.hidden_size, device=device))

In [75]:
X = torch.tensor([[1,2,3], [4,5,6]])
emb = nn.Embedding(8, 32)
emb(X).shape


torch.Size([2, 3, 32])

In [83]:
lstm = nn.LSTM(32, 64, num_layers=5)
o, s = lstm(emb(X))
print(o.shape)
print(s[0].shape)
print(s[1].shape)

torch.Size([2, 3, 64])
torch.Size([5, 3, 64])
torch.Size([5, 3, 64])


In [91]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return (torch.zeros(self.num_layers, 1, self.hidden_size, device=device),
                torch.zeros(self.num_layers, 1, self.hidden_size, device=device))


In [92]:
hidden_size = 256
encoder3 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder3 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder3, decoder3, 75000, print_every=5000)

1m 3s (- 14m 49s) (5000 6%) 3.3122
1m 59s (- 12m 56s) (10000 13%) 2.8866
2m 55s (- 11m 43s) (15000 20%) 2.6681
3m 52s (- 10m 38s) (20000 26%) 2.4932
4m 49s (- 9m 38s) (25000 33%) 2.3412
5m 47s (- 8m 40s) (30000 40%) 2.2159
6m 45s (- 7m 43s) (35000 46%) 2.1279
7m 42s (- 6m 44s) (40000 53%) 2.0191
8m 40s (- 5m 46s) (45000 60%) 1.9295
9m 37s (- 4m 48s) (50000 66%) 1.8326
10m 34s (- 3m 50s) (55000 73%) 1.7670
11m 30s (- 2m 52s) (60000 80%) 1.6856
12m 27s (- 1m 54s) (65000 86%) 1.6296
13m 23s (- 0m 57s) (70000 93%) 1.5687
14m 20s (- 0m 0s) (75000 100%) 1.4775


In [94]:
evaluateRandomly(encoder3, decoder3)

> мы здесь затем чтобы помочь тому .
= we re here to help tom .
< we re here to help tom help . <EOS>

> вы эксперт в этои области .
= you re an expert in the field .
< you re the the the the the . <EOS>

> боюсь этого будет недостаточно .
= i m afraid that won t be enough .
< i m afraid that ll t be . <EOS>

> вы добрыи человек .
= you re a kind man .
< you re a person person . <EOS>

> у меня от тебя мурашки по коже .
= you re giving me the creeps .
< you re not your your . . <EOS>

> они приятные люди .
= they re nice people .
< they re not the . <EOS>

> он без ума от нее .
= he s crazy about her .
< he s angry with her . <EOS>

> в октябре я еду в бостон .
= i m going to boston in october .
< i m going to boston next week . <EOS>

> мы остаемся .
= we re staying .
< we re going . <EOS>

> я так рад что ты позвонила .
= i m so glad you called .
< i m so glad you re pleased . <EOS>

